In [1]:
%matplotlib qt

import hyperspy.api as hs
import numpy as np
import matplotlib.pyplot as plt
import matplotlib as mpl

from pathlib import Path

gray_cmap = plt.colormaps.get('Greys')
gray_cmap.set_bad('lightblue')

In [12]:
SMALL_SIZE = 6
MEDIUM_SIZE = 8
BIGGER_SIZE = 10

plt.rc('font', size=SMALL_SIZE)          # controls default text sizes
plt.rc('axes', titlesize=SMALL_SIZE)     # fontsize of the axes title
plt.rc('axes', labelsize=MEDIUM_SIZE)    # fontsize of the x and y labels
plt.rc('xtick', labelsize=SMALL_SIZE)    # fontsize of the tick labels
plt.rc('ytick', labelsize=SMALL_SIZE)    # fontsize of the tick labels
plt.rc('legend', fontsize=SMALL_SIZE)    # legend fontsize
plt.rc('figure', titlesize=BIGGER_SIZE)  # fontsize of the figure title

## SVD
Plot the explained variance of the SVD decompositions

In [17]:
svd = {
    Path(r'C:\Users\emilc\OneDrive - NTNU\NORTEM\Data\PhaseMappingPaper\Data\Dataset A\datasetA_preprocessed_SVD1.npz'): 5,
    Path(r'C:\Users\emilc\OneDrive - NTNU\NORTEM\Data\PhaseMappingPaper\Data\Dataset A\datasetA_preprocessed_SVD2.npz'): 6,
    Path(r'C:\Users\emilc\OneDrive - NTNU\NORTEM\Data\PhaseMappingPaper\Data\Dataset B\datasetB_preprocessed_SVD1.npz'): 5,
    Path(r'C:\Users\emilc\OneDrive - NTNU\NORTEM\Data\PhaseMappingPaper\Data\Dataset B\datasetB_preprocessed_SVD2.npz'): 6
}

In [33]:
n = 21 #Number of components to show
components = np.arange(n)+1 #The component labelling.

#Plot/figure parameters
dpi=300
textwidth = 468 #pt
axes_per_line = 3
figwidth = textwidth/axes_per_line
pt2in = 0.01389
figsize = (figwidth*pt2in, figwidth*pt2in)

for svd_path in svd:
    s = np.load(svd_path, allow_pickle=True)
    threshold = svd[svd_path]
    
    signal_fmt={'marker': 'o', 'color': 'k', 'markerfacecolor': 'k', 'lw': 0, 'ms':4}
    noise_fmt={'marker':'o', 'color': 'k', 'markerfacecolor': 'w', 'lw': 0, 'ms':4}

    fig, ax = plt.subplots(figsize=figsize, dpi=dpi)
    ax.semilogy(components[:threshold], s['explained_variance_ratio'][:threshold], **signal_fmt)
    ax.semilogy(components[threshold:n], s['explained_variance_ratio'][threshold:n], **noise_fmt)
    ax.set_xlabel(f'Principal component number')
    ax.set_xticks(components[::5])
    ax.set_ylabel(f'Proportion of variance')
    plt.tight_layout()
    
    fig.savefig(svd_path.with_name(f'{svd_path.stem}_scree.png'))

## NMF
Plot the NMF components

In [34]:
nmf = [
    Path(r'C:\Users\emilc\OneDrive - NTNU\NORTEM\Data\PhaseMappingPaper\Data\Dataset A\datasetA_preprocessed_NMF1.npz'),
    Path(r'C:\Users\emilc\OneDrive - NTNU\NORTEM\Data\PhaseMappingPaper\Data\Dataset A\datasetA_preprocessed_NMF2.npz'),
    Path(r'C:\Users\emilc\OneDrive - NTNU\NORTEM\Data\PhaseMappingPaper\Data\Dataset B\datasetB_preprocessed_NMF1.npz'),
    Path(r'C:\Users\emilc\OneDrive - NTNU\NORTEM\Data\PhaseMappingPaper\Data\Dataset B\datasetB_preprocessed_NMF2.npz')
]

In [96]:
#Plot/figure parameters
dpi=300
textwidth = 468 #pt
axes_per_line = 3 #Number of axes on a line
figwidth = textwidth/axes_per_line
figheight = figwidth
pt2in = 0.01389
figsize = (figwidth*pt2in, figheight*pt2in)
inset_size = 0.25
for path in nmf:
    s = np.load(path, allow_pickle=True)
    factors = s['factors'].reshape(*s['original_shape'][2:], s['output_dimension'])
    loadings = s['loadings'].reshape(*s['original_shape'][:2], s['output_dimension'])
    for i in range(loadings.shape[-1]):
        fig = plt.figure(figsize=figsize, dpi=dpi)
        ax = fig.add_axes([0, 0, 1, 1], xticks = [], yticks = [], frameon=False)
        ax.imshow(loadings[:, :, i], cmap=gray_cmap)
        inset_ax = fig.add_axes([1-inset_size, 1-inset_size, inset_size, inset_size], xticks=[], yticks=[], frameon=False)
        inset_ax.imshow(factors[:, :, i], cmap=gray_cmap)
        fig.savefig(path.with_name(f'{path.stem}_{i}.png'))
        plt.close(fig)
    